# How to start

Before starting you must:
- Ensure that `scipp` and `mantid` are on your `PYTHONPATH`.
- Generate the `config.py` file using `make_config.py`. Refer to the `README.md` or `python make_config.py --help` for information.
- Install dependency `fabio`: `conda install fabio` (used for image loading)

For `scipp` and `mantid` follow instructions at: https://scipp.readthedocs.io/en/latest/getting-started/installation.html.

Converted to use scipp and notebook from [this file](https://git.esss.dk/testbeamline/gp2/blob/1c69213b1124982bbbe762da9c6c6457a49f2a92/reduce.py) by Dimitar Tasev on 2020-01-13

In [ ]:
# Some PYTHONPATH sanity checks

try:
    import scipp
    print("scipp found")
except ImportError as e:
    print("scipp is not available in the PYTHONPATH")
    raise e
    
try:
    import mantid
    print("mantid found")
except ImportError as e:
    print("mantid is not available in the PYTHONPATH")
    raise e
    
try:
    import config
    print("config found")
except ImportError as e:
    print("config is not available. Make sure you have generated it with `make_config.py`.")
    raise e

In [ ]:
import os

# Whether or not to do the plotting.
do_plots = False

# Specify where the data can be found, this should be the root folder
# containing the data_GP2 folder.
experiment_dir = config.script_root

# let's get the process started:
tofs_path = os.path.join(experiment_dir, 'data_GP2', 'metadata', 'GP2_BCC_time_values.txt')
sample_path = os.path.join(experiment_dir, 'data_GP2', 'Timeslices WFM BBC Steel')
ob_path = os.path.join(experiment_dir, 'data_GP2', 'Timeslices WFM Open Beam')
instrument_file = os.path.join(experiment_dir, 'IDF', 'V20_Definition_GP2.xml')

# defining grouping of 2D detector pixels
grouping_number = 3
nx_target = grouping_number
ny_target = grouping_number

# Rebin regions for each of the 5 frames
# in the format of [bin-start, bin-end, bin width].
# used to crop each one, before stitching them together
frame_parameters = [(15167, 23563, 64),
                    (24393, 32758, 64),
                    (33365, 40708, 64),
                    (41410, 48019, 64),
                    (49041, 55311, 64),
                    (56077, 59872, 64)]

# Used to shift the cropped frames so that their bins overlap 
# before summing them together into a single frame
frame_shift_increments = [-6630.0, -2420.0, -2253.0, -2095.0, -1946.0, -1810.0]
frame_shifts = [sum(frame_shift_increments[:i + 1]) for i in range(len(frame_shift_increments))]

# Used to rebin the summed frame in order to
# cut off frames that contain no data
rebin_parameters = (8500, 43000, 64)

In [ ]:
import fabio
import numpy as np
import glob
import sys
import csv

import scipp as sc
import numpy as np
from scipp import Dim

from mantid.simpleapi import *
from mantid.api import AlgorithmManager

In [ ]:
# Some helper functions

def read_x_values(tof_file):
    """
    Reads the TOF values from the CSV into a list
    """
    tof_values = list()
    with open(tof_file) as fh:
        csv_reader = csv.reader(fh, delimiter='\t')
        next(csv_reader, None)  # skip header
        for row in csv_reader:
            tof_values.append(float(row[1]))
    return tof_values


def _load_tiffs(tiff_dir):
    if not os.path.isdir(tiff_dir):
        raise RuntimeError(tiff_dir + " is not directory")
    stack = []
    path_length = len(tiff_dir) + 1
    filenames = sorted(glob.glob(tiff_dir + "/*.tiff"))
    nfiles = len(filenames)
    count = 0
    print(f"Loading {nfiles} files from '{tiff_dir}'")
    for filename in filenames:
        count += 1
        print('\r{0}: Image {1}, of {2}'.format(filename[path_length:], count, nfiles), end="")
        img = fabio.open(os.path.join(tiff_dir, filename))
        stack.append(np.flipud(img.data))

    return np.array(stack)

def tiffs_to_variable(tiff_dir):
    """
    Loads all tiff images from the directory into a scipp Variable.
    """
    stack = _load_tiffs(tiff_dir)
    data = stack.astype(np.float64).reshape(stack.shape[0], stack.shape[1]*stack.shape[2])
    return sc.Variable([Dim.Tof, Dim.Spectrum], 
                       values=data, variances=data)

from functools import reduce

def stitch(data_array, frame_parameters, rebin_parameters):
    """
    Stitches the 5 different frames data.
    
    It crops out each frame, then shifts it so that all frames align,
    and then rebins to the common bins used for all frames.
    """
    frames = []

    rebin_params = sc.Variable([Dim.Tof], values=np.arange(*rebin_parameters, dtype=np.float64))
    
    for i, (slice_bins, shift_parameter) in enumerate(zip(frame_parameters, frame_shifts)):
        bins = sc.Variable([Dim.Tof], values=np.arange(*slice_bins, dtype=np.float64))
        # Rebins the whole data to crop it to frame bins
        rebinned = sc.rebin(data_array, Dim.Tof, bins)
        # Shift the frame backwards to make all frames overlap
        rebinned.coords[Dim.Tof] += shift_parameter
        # Rebin to overarching coordinates so that the frame coordinates align
        rebinned = sc.rebin(rebinned, Dim.Tof, rebin_params)

        frames.append(rebinned)

    for f in frames[1:]:
        frames[0] += f

    return frames[0]

def zero_special_values(values):
    np.nan_to_num(values, copy=False)
    values[values == sys.float_info.max] = 0

In [ ]:
ds = sc.Dataset()

# Load X values from the TOF file
ds.coords[Dim.Tof] = sc.Variable([Dim.Tof], unit=sc.units.us, values=read_x_values(tofs_path))
ds.coords[Dim.Tof] *= 1e3

In [ ]:
# Load the data into the dataset
ds["sample"] = tiffs_to_variable(sample_path)
ds["reference"] = tiffs_to_variable(ob_path)

In [ ]:
sc.plot.plot(sc.sum(ds, Dim.Spectrum)) if do_plots else None

In [ ]:
# Adds a coordinate for the spectra
ds.coords[Dim.Spectrum] = sc.Variable([Dim.Spectrum], values=np.arange(ds["sample"].shape[1]))
ds

In [ ]:
stitched = sc.Dataset(coords={Dim.Tof: sc.Variable([Dim.Tof], values=np.arange(*rebin_parameters, dtype=np.float64))})

stitched["sample"] = stitch(ds["sample"], frame_parameters, rebin_parameters)
stitched["reference"] = stitch(ds["reference"], frame_parameters, rebin_parameters)

stitched

In [ ]:
def make_detector_groups(nx_original, ny_original, nx_target, ny_target):
    element_width_x = nx_original // nx_target
    element_width_y = ny_original // ny_target
    
    # To contain our new spectra mappings
    grid = np.zeros((nx_original, ny_original), dtype=np.float64)

    for i in range(0, nx_target):
        x_start = i * element_width_x
        x_end = (i + 1) * element_width_x
        
        for j in range(0, ny_target):
            y_start = j * element_width_y
            y_end = (j + 1) * element_width_y

            vals = np.full((element_width_x, element_width_y), i + j * nx_target, dtype=np.float64)
            grid[x_start:x_end, y_start:y_end] = vals
    
    return sc.Variable([Dim.Spectrum], values=grid.ravel())


stitched.labels["detector_mapping"] = make_detector_groups(324, 324, nx_target, ny_target)
stitched.labels["detector_mapping"]

In [ ]:
stitched["normalized"] = stitched["sample"] / stitched["reference"]

zero_special_values(stitched["normalized"].values)

In [ ]:
dm1d = sc.groupby(stitched["normalized"], "detector_mapping", Dim.Row)
stitched["normalized_grpd"] = dm1d.sum(Dim.Spectrum)

In [ ]:
# Plot the first group
sc.plot.plot(sc.sum(stitched["normalized_grpd"], Dim.Row)) if do_plots else None

In [ ]:
# Adds the component info needed for converting units
sc.compat.mantid.load_component_info(stitched, instrument_file)
stitched

In [ ]:
# Note: this cell is currently a workaround until 
# https://github.com/scipp/scipp/issues/823 is done.
# Then it should be possible to group the positions label directly.

# makes the position a DataConstProxy otherwise groupby won't take it
stitched["position"] = stitched.labels["position"]
dm1d = sc.groupby(stitched["position"], "detector_mapping", Dim.Row)
position = dm1d.mean(Dim.Spectrum)

# can't do stitched.labels["position"] = position because Labels won't take a DataArray
# also can't do stitched.labels["position"] = sc.Variable(position) because then sc.convert 
# thinks stitched.labels["position"] is dimensionless (as it's actually still a DataArray)

pos = np.zeros((position.shape[0], 3))
for i, val in enumerate(position.values):
    pos[i, :] = val

# finally add it back!
stitched.labels["position"] = sc.Variable(position.dims, pos, unit=sc.units.m, dtype=position.dtype)

In [ ]:
stitched["normalized_wl"] = sc.neutron.convert(stitched["normalized_grpd"], Dim.Tof, Dim.Wavelength)
stitched

In [ ]:
sc.plot.plot(stitched["normalized_wl"][Dim.Row, 0]) if do_plots else None

In [ ]:
x_coords = stitched["normalized_wl"].coords[stitched["normalized_wl"].dims[0]]
x_dim = stitched["normalized_wl"].dims[0]
x = x_coords.values

# Mantid expects the data in a different shape
# which is spectrum as outer-most dimension.
y = np.transpose(stitched["normalized_wl"].values)

e = np.transpose(stitched["normalized_wl"].variances)

ws = sc.compat.mantid.to_workspace_2d(x, y, e, x_dim, instrument_file)
ws

In [ ]:
# If you want to save the workspace uncomment this line
SaveNexusESS(ws, "scipp_normalized_wl.nxs")

In [ ]:
fit_ds = sc.compat.mantid.fit(ws, 
                     function='name=LinearBackground,A0=5000,A1=0;name=UserFunction,Formula=h*erfc(a*(x-x0)),h=5000,a=-0.5,x0=4',
                    workspace_index=2, start_x=3.6, end_x=4.4)

In [ ]:
fit_ds

In [ ]:
sc.plot.plot(fit_ds["workspace"].value, collapse=Dim.Wavelength)